# Characterizing CpG Methylation (union bedgraphs with 5x data)

In this notebook, general methylation landscapes in *Montipora capitata* and *Pocillopora acuta* will be characterized based on WGSB, RRBS, and MBD-BSseq data. I will also assess CG motif overlaps with various genome feature tracks to understand where methylation may occur across the genome. I will use [union bedgraphs](https://gannet.fish.washington.edu/seashell/bu-github/Meth_Compare/analyses/10-unionbedg/) with 5x data.

1. Download union bedgraphs and format for downstream analyses
2. Characterize methylation for each CpG dinucleotide
3. Characterize genomic locations of methylated CpGs, sparsely methylated CpGs, and unmethylated CpGs for each sequencing type

## 0. Set working directory and install programs

In [1]:
!pwd

/Users/yaamini/Documents/Meth_Compare/scripts


In [2]:
cd ../analyses/

/Users/yaamini/Documents/Meth_Compare/analyses


In [3]:
#!mkdir Characterizing-CpG-Methylation-5x-Union

mkdir: Characterizing-CpG-Methylation-5x-Union: File exists


In [4]:
cd Characterizing-CpG-Methylation-5x-Union/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x-Union


In [5]:
#Install pandas for this notebook
import pandas as pd
print(pd.__version__)

0.18.1


## *M. capitata*

In [6]:
#Make a directory for Mcap output
!mkdir Mcap

In [7]:
cd Mcap/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x-Union/Mcap


### 1. Format data

#### 1a. Download bedgraph

In [8]:
#Download Mcap 5x union bedgraph
!wget https://gannet.fish.washington.edu/seashell/bu-github/Meth_Compare/analyses/10-unionbedg/Mcap_union_5x.bedgraph

--2020-05-19 21:52:34--  https://gannet.fish.washington.edu/seashell/bu-github/Meth_Compare/analyses/10-unionbedg/Mcap_union_5x.bedgraph
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862402537 (822M)
Saving to: ‘Mcap_union_5x.bedgraph’

Mcap_union_5x.bedgr 100%[===================>] 822.45M  90.9MB/s    in 10s     

2020-05-19 21:52:45 (81.3 MB/s) - ‘Mcap_union_5x.bedgraph’ saved [862402537/862402537]



In [9]:
#Check downloaded file
#WGBS: 10-12
#RRBS: 14-16
#MBD-BS: 16-18
!tail Mcap_union_5x.bedgraph

3043	18674	18676	12.500000	N/A	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
3043	18688	18690	7.692308	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	N/A
3043	18720	18722	8.333333	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	0.000000
3043	18734	18736	8.333333	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	0.000000
3043	18740	18742	0.000000	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	0.000000
3043	18755	18757	11.111111	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	0.000000
3043	18784	18786	20.000000	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	0.000000
3043	18801	18803	0.000000	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	0.000000
3043	18822	18824	N/A	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	N/A
3043	18827	18829	N/A	0.000000	0.000000	N/A	N/A	N/A	N/A	0.000000	N/A


#### 1b. Manipulate with `pandas`

In [10]:
#Import union data into pandas
#Check head
df = pd.read_table("Mcap_union_5x.bedgraph")
df.head(5)

,chrom,start,end,10,11,12,13,14,15,16,17,18
0,1,3493,3495,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN
1,1,3518,3520,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,NaN,NaN
2,1,3727,3729,NaN,NaN,NaN,0.0,0.0,8.695652,NaN,NaN,NaN
3,1,3752,3754,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,NaN,NaN
4,1,3757,3759,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,NaN,NaN


In [11]:
#Average the first three columns for WGBS information and save as a new column
#Average the middle three columns for RRBS information and save as a new column
#Average the last three columns for MBD-BS information and save as a new column
#Check output
df['WGBS'] = df[['10', '11', "12"]].mean(axis=1)
df['RRBS'] = df[['13', '14', "15"]].mean(axis=1)
df['MBD-BS'] = df[['16', '17', "18"]].mean(axis=1)
df.tail(10)

,chrom,start,end,10,11,12,13,14,15,16,17,18,WGBS,RRBS,MBD-BS
13340258,3043,18674,18676,12.500000,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,6.250000,NaN,NaN
13340259,3043,18688,18690,7.692308,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,2.564103,NaN,0.0
13340260,3043,18720,18722,8.333333,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,2.777778,NaN,0.0
13340261,3043,18734,18736,8.333333,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,2.777778,NaN,0.0
13340262,3043,18740,18742,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,0.0
13340263,3043,18755,18757,11.111111,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,3.703704,NaN,0.0
13340264,3043,18784,18786,20.000000,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,6.666667,NaN,0.0
13340265,3043,18801,18803,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0
13340266,3043,18822,18824,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,0.0
13340267,3043,18827,18829,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.000000,NaN,0.0


In [12]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("Mcap_union_5x-averages.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

#### 1c. Separate methods into new bedgraphs

In [13]:
#Check pandas manipulations
!head Mcap_union_5x-averages.bedgraph

	chrom	start	end	10	11	12	13	14	15	16	17	18	WGBS	RRBS	MBD-BS
0	1	3493	3495	N/A	N/A	N/A	0.0	N/A	0.0	N/A	N/A	N/A	N/A	0.0	N/A
1	1	3518	3520	N/A	N/A	N/A	0.0	N/A	0.0	N/A	N/A	N/A	N/A	0.0	N/A
2	1	3727	3729	N/A	N/A	N/A	0.0	0.0	8.695652	N/A	N/A	N/A	N/A	2.898550666666667	N/A
3	1	3752	3754	N/A	N/A	N/A	0.0	0.0	0.0	N/A	N/A	N/A	N/A	0.0	N/A
4	1	3757	3759	N/A	N/A	N/A	0.0	0.0	0.0	N/A	N/A	N/A	N/A	0.0	N/A
5	1	3770	3772	N/A	N/A	N/A	0.0	0.0	0.0	N/A	N/A	N/A	N/A	0.0	N/A
6	1	4062	4064	N/A	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
7	1	4069	4071	N/A	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
8	1	4077	4079	N/A	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A


In [14]:
#Remove header
#Keep chr, start, end, and WGBS average (col 2-4, 13)
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Mcap_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $14}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Mcap_union_5x-averages-WGBS.bedgraph

In [15]:
#Check output: chr, start, end, % meth
!head Mcap_union_5x-averages-WGBS.bedgraph
!wc -l Mcap_union_5x-averages-WGBS.bedgraph

1	4062	4064	0.0
1	4069	4071	0.0
1	4077	4079	0.0
1	4086	4088	0.0
1	4146	4148	0.0
1	4150	4152	0.0
1	4155	4157	0.0
1	4172	4174	0.0
1	4184	4186	0.0
1	4190	4192	16.666667
 11509837 Mcap_union_5x-averages-WGBS.bedgraph


In [16]:
#Remove header
#Keep chr, start, end, and RRBS average
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Mcap_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $15}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Mcap_union_5x-averages-RRBS.bedgraph

In [17]:
#Check output: chr, start, end, % meth
!head Mcap_union_5x-averages-RRBS.bedgraph
!wc -l Mcap_union_5x-averages-RRBS.bedgraph

1	3493	3495	0.0
1	3518	3520	0.0
1	3727	3729	2.898550666666667
1	3752	3754	0.0
1	3757	3759	0.0
1	3770	3772	0.0
1	11876	11878	0.0
1	11887	11889	0.0
1	11894	11896	0.0
1	11941	11943	0.0
 3981450 Mcap_union_5x-averages-RRBS.bedgraph


In [18]:
#Remove header
#Keep chr, start, end, and MBD-BS average
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Mcap_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $16}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Mcap_union_5x-averages-MBDBS.bedgraph

In [19]:
#Check output: chr, start, end, % meth
!head Mcap_union_5x-averages-MBDBS.bedgraph
!wc -l Mcap_union_5x-averages-MBDBS.bedgraph

1	5228	5230	0.0
1	5243	5245	0.0
1	5247	5249	0.0
1	5296	5298	0.0
1	8113	8115	20.0
1	59438	59440	100.0
1	77096	77098	0.0
1	77145	77147	0.0
1	77151	77153	0.0
1	77179	77181	0.0
  866555 Mcap_union_5x-averages-MBDBS.bedgraph


In [20]:
!find *averages-*bedgraph

Mcap_union_5x-averages-MBDBS.bedgraph
Mcap_union_5x-averages-RRBS.bedgraph
Mcap_union_5x-averages-WGBS.bedgraph


In [21]:
!wc -l *averages-*bedgraph > Mcap_union_5x-averages-counts.txt

### 2. Characterize methylation for each CpG dinucleotide

- Methylated: > 50% methylation
- Sparsely methylated: 10-50% methylation
- Unmethylated: < 10% methylation

##### Methylated loci

In [23]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-Meth
done

In [24]:
!head *-Meth

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth <==
1 59438 59440 100.0
1 106173 106175 100.0
1 106202 106204 100.0
1 344031 344033 50.0
1 344044 344046 60.0
1 446326 446328 80.0
1 446344 446346 100.0
1 446367 446369 100.0
1 446376 446378 100.0
1 786125 786127 60.0

==> Mcap_union_5x-averages-RRBS.bedgraph-Meth <==
1 32228 32230 50.413223
1 58618 58620 95.65826333333332
1 58745 58747 96.819728
1 58764 58766 99.16666666666667
1 58792 58794 83.42830033333334
1 66041 66043 100.0
1 66050 66052 100.0
1 66339 66341 88.888889
1 66345 66347 77.777778
1 66354 66356 77.777778

==> Mcap_union_5x-averages-WGBS.bedgraph-Meth <==
1 4948 4950 50.0
1 4967 4969 50.0
1 4986 4988 50.0
1 57065 57067 80.0
1 58609 58611 100.0
1 58618 58620 100.0
1 58745 58747 100.0
1 59207 59209 100.0
1 59277 59279 100.0
1 59393 59395 100.0


In [25]:
!wc -l *-Meth

  148321 Mcap_union_5x-averages-MBDBS.bedgraph-Meth
  329361 Mcap_union_5x-averages-RRBS.bedgraph-Meth
 1350936 Mcap_union_5x-averages-WGBS.bedgraph-Meth
 1828618 total


In [26]:
!wc -l *-Meth > Mcap_union_5x-Meth-counts.txt

##### Sparsely methylated loci

In [27]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 < 50) { print $1, $2, $3, $4}}' ${f} \
    | awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
    > ${f}-sparseMeth
done

In [28]:
!head *-sparseMeth

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth <==
1 8113 8115 20.0
1 211907 211909 40.0
1 217198 217200 14.285714000000002
1 234158 234160 14.285714000000002
1 234196 234198 12.5
1 244563 244565 20.0
1 269174 269176 16.666667
1 269178 269180 16.666667
1 269182 269184 16.666667
1 277994 277996 16.666667

==> Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth <==
1 15092 15094 30.0
1 21739 21741 13.636364000000002
1 34139 34141 11.764706
1 42261 42263 10.539216
1 45163 45165 10.31746
1 48370 48372 14.285714000000002
1 87492 87494 33.333333
1 89011 89013 14.285714000000002
1 101503 101505 17.380952
1 101545 101547 23.3333335

==> Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth <==
1 4190 4192 16.666667
1 4891 4893 33.333333
1 4910 4912 28.571429
1 4929 4931 16.6666665
1 5005 5007 28.571429
1 5024 5026 40.0
1 5151 5153 20.0
1 5160 5162 16.666667
1 5228 5230 11.111111
1 6282 6284 11.111111


In [29]:
!wc -l *-sparseMeth

  103713 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth
  220277 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth
 1155033 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth
 1479023 total


In [30]:
!wc -l *-sparseMeth > Mcap_union_5x-sparseMeth-counts.txt

##### Unmethylated loci

In [31]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-unMeth
done

In [32]:
!head *-unMeth

==> Mcap_union_5x-averages-MBDBS.bedgraph-unMeth <==
1 5228 5230 0.0
1 5243 5245 0.0
1 5247 5249 0.0
1 5296 5298 0.0
1 77096 77098 0.0
1 77145 77147 0.0
1 77151 77153 0.0
1 77179 77181 0.0
1 81812 81814 0.0
1 81817 81819 0.0

==> Mcap_union_5x-averages-RRBS.bedgraph-unMeth <==
1 3493 3495 0.0
1 3518 3520 0.0
1 3727 3729 2.898550666666667
1 3752 3754 0.0
1 3757 3759 0.0
1 3770 3772 0.0
1 11876 11878 0.0
1 11887 11889 0.0
1 11894 11896 0.0
1 11941 11943 0.0

==> Mcap_union_5x-averages-WGBS.bedgraph-unMeth <==
1 4062 4064 0.0
1 4069 4071 0.0
1 4077 4079 0.0
1 4086 4088 0.0
1 4146 4148 0.0
1 4150 4152 0.0
1 4155 4157 0.0
1 4172 4174 0.0
1 4184 4186 0.0
1 5043 5045 0.0


In [33]:
!wc -l *-unMeth

  614521 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth
 3431812 Mcap_union_5x-averages-RRBS.bedgraph-unMeth
 9003868 Mcap_union_5x-averages-WGBS.bedgraph-unMeth
 13050201 total


In [34]:
!wc -l *-unMeth > Mcap_union_5x-unMeth-counts.txt

### 3. Characterize genomic locations of CpGs

#### 3a. Create BEDfiles

In [35]:
%%bash

for f in *averages-*bedgraph*
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  866555 Mcap_union_5x-averages-MBDBS.bedgraph.bed
  148321 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed
  103713 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed
  614521 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed
 3981450 Mcap_union_5x-averages-RRBS.bedgraph.bed
  329361 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed
  220277 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed
 3431812 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed
 11509837 Mcap_union_5x-averages-WGBS.bedgraph.bed
 1350936 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed
 1155033 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed
 9003868 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed


In [36]:
#Confirm file creation
!head Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed

1	5228	5230
1	5243	5245
1	5247	5249
1	5296	5298
1	77096	77098
1	77145	77147
1	77151	77153
1	77179	77181
1	81812	81814
1	81817	81819


#### 3b. Genes

In [37]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.gene.gff \
  > ${f}-mcGenes
done

In [38]:
#Check output
!head *mcGenes

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcGenes <==
1	59438	59440	1	AUGUSTUS	gene	58322	62557	1	-	.	g21535
1	106173	106175	1	AUGUSTUS	gene	100215	109729	0.99	-	.	g21538
1	106202	106204	1	AUGUSTUS	gene	100215	109729	0.99	-	.	g21538
1	344031	344033	1	AUGUSTUS	gene	332282	348965	0.61	-	.	g21555
1	344044	344046	1	AUGUSTUS	gene	332282	348965	0.61	-	.	g21555
1	786125	786127	1	AUGUSTUS	gene	785899	786207	0.98	-	.	g21598
1	786144	786146	1	AUGUSTUS	gene	785899	786207	0.98	-	.	g21598
1	786151	786153	1	AUGUSTUS	gene	785899	786207	0.98	-	.	g21598
1	879915	879917	1	AUGUSTUS	gene	875739	924977	0.37	+	.	g21603
1	883893	883895	1	AUGUSTUS	gene	875739	924977	0.37	+	.	g21603

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcGenes <==
1	323095	323097	1	AUGUSTUS	gene	323002	331686	1	-	.	g21554
1	328382	328384	1	AUGUSTUS	gene	323002	331686	1	-	.	g21554
1	328386	328388	1	AUGUSTUS	gene	323002	331686	1	-	.	g21554
1	330194	330196	1	AUGUSTUS	gene	323002	331686	1	-	.	g21554
1	330197	330199	1	

In [39]:
#Count number of overlaps
!wc -l *mcGenes

   67258 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcGenes
   35276 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcGenes
  207003 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcGenes
  309537 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcGenes
  156521 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcGenes
   77322 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcGenes
 1073239 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcGenes
 1307082 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcGenes
  685807 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcGenes
  400637 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcGenes
 3054154 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcGenes
 4140598 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcGenes
 11514434 total


In [40]:
!wc -l *mcGenes > Mcap_union_5x-mcGenes-counts.txt

#### 3c. Coding Sequences (CDS)

In [41]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.CDS.gff \
  > ${f}-mcCDS
done

In [42]:
#Check output
!head *mcCDS

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcCDS <==
1	59438	59440	1	AUGUSTUS	CDS	58322	59506	1	-	0	transcript_id "g21535.t1"; gene_id "g21535";
1	786125	786127	1	AUGUSTUS	CDS	785899	786207	0.98	-	0	transcript_id "g21598.t1"; gene_id "g21598";
1	786144	786146	1	AUGUSTUS	CDS	785899	786207	0.98	-	0	transcript_id "g21598.t1"; gene_id "g21598";
1	786151	786153	1	AUGUSTUS	CDS	785899	786207	0.98	-	0	transcript_id "g21598.t1"; gene_id "g21598";
1	1263040	1263042	1	AUGUSTUS	CDS	1262915	1264300	1	-	0	transcript_id "g21628.t1"; gene_id "g21628";
1	1409642	1409644	1	AUGUSTUS	CDS	1409580	1409782	1	-	2	transcript_id "g21635.t1"; gene_id "g21635";
1	1409734	1409736	1	AUGUSTUS	CDS	1409580	1409782	1	-	2	transcript_id "g21635.t1"; gene_id "g21635";
1	1543924	1543926	1	AUGUSTUS	CDS	1543864	1543969	1	+	0	transcript_id "g21645.t1"; gene_id "g21645";
1	1601051	1601053	1	AUGUSTUS	CDS	1600981	1601078	0.86	-	2	transcript_id "g21648.t1"; gene_id "g21648";
1	1641103	1641105	1	AUGUSTUS	CDS	1640990	164360

In [43]:
#Count number of overlaps
!wc -l *mcCDS

   18518 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcCDS
   13110 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcCDS
   74148 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcCDS
  105776 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcCDS
   22660 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcCDS
   16593 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcCDS
  205283 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcCDS
  244536 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcCDS
  139018 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcCDS
  103408 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcCDS
  753113 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcCDS
  995539 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcCDS
 2691702 total


In [44]:
!wc -l *mcCDS > Mcap_union_5x-mcCDS-counts.txt

#### 3d. Introns

In [45]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.intron.gff \
  > ${f}-mcIntrons
done

In [46]:
#Check output
!head *mcIntrons

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcIntrons <==
1	106173	106175	1	AUGUSTUS	intron	104815	109637	1	-	.	transcript_id "g21538.t1"; gene_id "g21538";
1	106202	106204	1	AUGUSTUS	intron	104815	109637	1	-	.	transcript_id "g21538.t1"; gene_id "g21538";
1	344031	344033	1	AUGUSTUS	intron	341159	344880	1	-	.	transcript_id "g21555.t1"; gene_id "g21555";
1	344044	344046	1	AUGUSTUS	intron	341159	344880	1	-	.	transcript_id "g21555.t1"; gene_id "g21555";
1	879915	879917	1	AUGUSTUS	intron	879326	880196	1	+	.	transcript_id "g21603.t1"; gene_id "g21603";
1	883893	883895	1	AUGUSTUS	intron	883512	889480	1	+	.	transcript_id "g21603.t1"; gene_id "g21603";
1	982886	982888	1	AUGUSTUS	intron	980695	983102	1	-	.	transcript_id "g21609.t1"; gene_id "g21609";
1	1243019	1243021	1	AUGUSTUS	intron	1242686	1245119	1	-	.	transcript_id "g21628.t1"; gene_id "g21628";
1	1259506	1259508	1	AUGUSTUS	intron	1259175	1261381	1	-	.	transcript_id "g21628.t1"; gene_id "g21628";
1	1259529	1259531	1	AUGUSTUS	intron	

In [47]:
#Count number of overlaps
!wc -l *mcIntrons

   48781 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcIntrons
   22188 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcIntrons
  133009 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcIntrons
  203978 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcIntrons
  133901 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcIntrons
   60756 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcIntrons
  868445 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcIntrons
 1063102 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcIntrons
  547317 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcIntrons
  297449 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcIntrons
 2303015 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcIntrons
 3147781 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcIntrons
 8829722 total


In [48]:
!wc -l *mcIntrons > Mcap_union_5x-mcIntrons-counts.txt

#### 3e. Flanking regions

In [49]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.gff \
  > ${f}-mcFlanks
done

In [50]:
#Check output
!head *mcFlanks

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanks <==
1	789213	789215	1	AUGUSTUS	gene	788380	789379	0.68	-	.	g21600
1	2070697	2070699	1	AUGUSTUS	gene	2070531	2071530	0.88	-	.	g21691
1	2070732	2070734	1	AUGUSTUS	gene	2070531	2071530	0.88	-	.	g21691
2	126187	126189	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126190	126192	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126197	126199	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126199	126201	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	173804	173806	2	AUGUSTUS	gene	173011	174010	0.32	+	.	g21795
2	173810	173812	2	AUGUSTUS	gene	173011	174010	0.32	+	.	g21795
2	197311	197313	2	AUGUSTUS	gene	196829	197828	0.17	+	.	g21798

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanks <==
1	217198	217200	1	AUGUSTUS	gene	217011	218010	0.95	-	.	g21549
1	376177	376179	1	AUGUSTUS	gene	375372	376371	0.32	-	.	g21558
1	458648	458650	1	AUGUSTUS	gene	458012	459011	0.87	-	.	g21567
1	618190	618192	1	AUGUSTUS	gene	617737	618736	1	+	.	g21

In [51]:
#Count number of overlaps
!wc -l *mcFlanks

   19297 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanks
   12581 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanks
   65423 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcFlanks
   97301 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcFlanks
   37968 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcFlanks
   27669 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcFlanks
  361553 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcFlanks
  427190 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcFlanks
  151474 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcFlanks
  142393 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcFlanks
  985243 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcFlanks
 1279110 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcFlanks
 3607202 total


In [52]:
!wc -l *mcFlanks > Mcap_union_5x-mcFlanks-counts.txt

#### 3f. Upstream flanking regions

In [53]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.Upstream.gff \
  > ${f}-mcFlanksUpstream
done

In [54]:
#Check output
!head *mcFlanksUpstream

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanksUpstream <==
2	126187	126189	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126190	126192	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126197	126199	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	126199	126201	2	AUGUSTUS	gene	125245	126244	0.6	-	.	g21784
2	173804	173806	2	AUGUSTUS	gene	173011	174010	0.32	+	.	g21795
2	173810	173812	2	AUGUSTUS	gene	173011	174010	0.32	+	.	g21795
2	389824	389826	2	AUGUSTUS	gene	389559	390558	0.83	-	.	g21842
2	420160	420162	2	AUGUSTUS	gene	419558	420557	0.88	+	.	g21852
2	445154	445156	2	AUGUSTUS	gene	445143	446142	0.63	-	.	g21856
2	445170	445172	2	AUGUSTUS	gene	445143	446142	0.63	-	.	g21856

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanksUpstream <==
1	376177	376179	1	AUGUSTUS	gene	375372	376371	0.32	-	.	g21558
1	618190	618192	1	AUGUSTUS	gene	617737	618736	1	+	.	g21582
1	618205	618207	1	AUGUSTUS	gene	617737	618736	1	+	.	g21582
1	726420	726422	1	AUGUSTUS	gene	726311	726537	0.72	-	

In [55]:
#Count number of overlaps
!wc -l *mcFlanksUpstream

   10087 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanksUpstream
    6951 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanksUpstream
   35949 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcFlanksUpstream
   52987 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcFlanksUpstream
   19889 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcFlanksUpstream
   14562 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcFlanksUpstream
  201313 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcFlanksUpstream
  235764 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcFlanksUpstream
   77156 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcFlanksUpstream
   74507 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcFlanksUpstream
  533799 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcFlanksUpstream
  685462 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcFlanksUpstream
 1948426 total


In [56]:
!wc -l *mcFlanksUpstream > Mcap_union_5x-mcFlanksUpstream-counts.txt

#### 3g. Downstream flanking regions

In [57]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.Downstream.gff \
  > ${f}-mcFlanksDownstream
done

In [58]:
#Check output
!head *mcFlanksDownstream

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanksDownstream <==
1	789213	789215	1	AUGUSTUS	gene	788380	789379	0.68	-	.	g21600
1	2070697	2070699	1	AUGUSTUS	gene	2070531	2071530	0.88	-	.	g21691
1	2070732	2070734	1	AUGUSTUS	gene	2070531	2071530	0.88	-	.	g21691
2	197311	197313	2	AUGUSTUS	gene	196829	197828	0.17	+	.	g21798
2	197321	197323	2	AUGUSTUS	gene	196829	197828	0.17	+	.	g21798
2	197327	197329	2	AUGUSTUS	gene	196829	197828	0.17	+	.	g21798
2	260911	260913	2	AUGUSTUS	gene	259987	260986	0.6	-	.	g21809
2	301621	301623	2	AUGUSTUS	gene	301474	302473	0.73	+	.	g21817
2	330280	330282	2	AUGUSTUS	gene	329400	330399	1	+	.	g21824
2	330291	330293	2	AUGUSTUS	gene	329400	330399	1	+	.	g21824

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanksDownstream <==
1	217198	217200	1	AUGUSTUS	gene	217011	218010	0.95	-	.	g21549
1	458648	458650	1	AUGUSTUS	gene	458012	459011	0.87	-	.	g21567
1	743459	743461	1	AUGUSTUS	gene	742708	743707	0.98	-	.	g21595
1	778795	778797	1	AUGUSTUS	gene	777826

In [59]:
#Count number of overlaps
!wc -l *mcFlanksDownstream

    9210 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcFlanksDownstream
    5630 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcFlanksDownstream
   29474 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcFlanksDownstream
   44314 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcFlanksDownstream
   18079 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcFlanksDownstream
   13107 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcFlanksDownstream
  160240 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcFlanksDownstream
  191426 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcFlanksDownstream
   74318 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcFlanksDownstream
   67886 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcFlanksDownstream
  451444 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcFlanksDownstream
  593648 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcFlanksDownstream
 1658776 total


In [60]:
!wc -l *mcFlanksDownstream > Mcap_union_5x-mcFlanksDownstream-counts.txt

#### 3h. Intergenic

In [61]:
%%bash 

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.intergenic.bed \
  > ${f}-mcIntergenic
done

In [62]:
#Check output
!head *mcIntergenic

==> Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcIntergenic <==
1	446326	446328	1	444635	458011
1	446344	446346	1	444635	458011
1	446367	446369	1	444635	458011
1	446376	446378	1	444635	458011
1	1002973	1002975	1	990352	1025880
1	1006917	1006919	1	990352	1025880
1	1006924	1006926	1	990352	1025880
1	1343240	1343242	1	1339740	1346874
1	1343249	1343251	1	1339740	1346874
1	1343263	1343265	1	1339740	1346874

==> Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcIntergenic <==
1	8113	8115	1	0	17386
1	211907	211909	1	187520	217010
1	234158	234160	1	219415	236954
1	234196	234198	1	219415	236954
1	244563	244565	1	239956	306771
1	269174	269176	1	239956	306771
1	269178	269180	1	239956	306771
1	269182	269184	1	239956	306771
1	277994	277996	1	239956	306771
1	284269	284271	1	239956	306771

==> Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcIntergenic <==
1	5228	5230	1	0	17386
1	5243	5245	1	0	17386
1	5247	5249	1	0	17386
1	5296	5298	1	0	17386
1	192753	192755	1	187520	217010
1	210921	210923

In [63]:
#Count number of overlaps
!wc -l *mcIntergenic

   65566 Mcap_union_5x-averages-MBDBS.bedgraph-Meth.bed-mcIntergenic
   57824 Mcap_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-mcIntergenic
  348119 Mcap_union_5x-averages-MBDBS.bedgraph-unMeth.bed-mcIntergenic
  471509 Mcap_union_5x-averages-MBDBS.bedgraph.bed-mcIntergenic
  140391 Mcap_union_5x-averages-RRBS.bedgraph-Meth.bed-mcIntergenic
  119284 Mcap_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-mcIntergenic
 2023518 Mcap_union_5x-averages-RRBS.bedgraph-unMeth.bed-mcIntergenic
 2283193 Mcap_union_5x-averages-RRBS.bedgraph.bed-mcIntergenic
  537847 Mcap_union_5x-averages-WGBS.bedgraph-Meth.bed-mcIntergenic
  630806 Mcap_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-mcIntergenic
 5040188 Mcap_union_5x-averages-WGBS.bedgraph-unMeth.bed-mcIntergenic
 6208841 Mcap_union_5x-averages-WGBS.bedgraph.bed-mcIntergenic
 17927086 total


In [64]:
!wc -l *mcIntergenic > Mcap_union_5x-mcIntergenic-counts.txt

## *P. acuta*

In [65]:
cd ..

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x-Union


In [66]:
#Make a directory for Pact output
!mkdir Pact

In [67]:
cd Pact/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x-Union/Pact


#### 1a. Download bedgraph

In [68]:
#Download Mcap 5x union bedgraph
!wget https://gannet.fish.washington.edu/seashell/bu-github/Meth_Compare/analyses/10-unionbedg/Pact_union_5x.bedgraph

--2020-05-19 22:34:21--  https://gannet.fish.washington.edu/seashell/bu-github/Meth_Compare/analyses/10-unionbedg/Pact_union_5x.bedgraph
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1212075709 (1.1G)
Saving to: ‘Pact_union_5x.bedgraph’

Pact_union_5x.bedgr 100%[===================>]   1.13G  67.6MB/s    in 14s     

2020-05-19 22:34:35 (84.8 MB/s) - ‘Pact_union_5x.bedgraph’ saved [1212075709/1212075709]



In [69]:
#Check downloaded file
#WGBS: 1-3
#RRBS: 4-6
#MBD-BS: 7-9
!tail Pact_union_5x.bedgraph

scaffold168460_cov188	280	282	N/A	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168461_cov203	69	71	N/A	N/A	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168461_cov203	142	144	N/A	N/A	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168461_cov203	148	150	N/A	N/A	12.500000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168463_cov229	14	16	N/A	10.000000	N/A	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168463_cov229	95	97	N/A	5.555556	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168463_cov229	106	108	N/A	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168465_cov225	45	47	N/A	0.000000	N/A	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168465_cov225	48	50	N/A	0.000000	N/A	N/A	N/A	N/A	N/A	N/A	N/A
scaffold168465_cov225	59	61	N/A	0.000000	0.000000	N/A	N/A	N/A	N/A	N/A	N/A


#### 1b. Manipulate with `pandas`

In [70]:
#Import union data into pandas
#Check head
df = pd.read_table("Pact_union_5x.bedgraph")
df.head(5)

,chrom,start,end,1,2,3,4,5,6,7,8,9
0,scaffold1_cov55,49,51,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,scaffold1_cov55,84,86,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,scaffold1_cov55,92,94,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,scaffold1_cov55,102,104,16.666667,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,scaffold1_cov55,105,107,0.000000,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
#Average the first three columns for WGBS information and save as a new column
#Average the middle three columns for RRBS information and save as a new column
#Average the last three columns for MBD-BS information and save as a new column
#Check output
df['WGBS'] = df[['1', '2', "3"]].mean(axis=1)
df['RRBS'] = df[['4', '5', "6"]].mean(axis=1)
df['MBD-BS'] = df[['7', '8', "9"]].mean(axis=1)
df.tail(10)

,chrom,start,end,1,2,3,4,5,6,7,8,9,WGBS,RRBS,MBD-BS
15758665,scaffold168460_cov188,280,282,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758666,scaffold168461_cov203,69,71,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758667,scaffold168461_cov203,142,144,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758668,scaffold168461_cov203,148,150,NaN,NaN,12.5,NaN,NaN,NaN,NaN,NaN,NaN,12.500000,NaN,NaN
15758669,scaffold168463_cov229,14,16,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN
15758670,scaffold168463_cov229,95,97,NaN,5.555556,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.777778,NaN,NaN
15758671,scaffold168463_cov229,106,108,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758672,scaffold168465_cov225,45,47,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758673,scaffold168465_cov225,48,50,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
15758674,scaffold168465_cov225,59,61,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN


In [72]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("Pact_union_5x-averages.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

#### 1c. Separate methods into new bedgraphs

In [73]:
#Check pandas manipulations
!head Pact_union_5x-averages.bedgraph

	chrom	start	end	1	2	3	4	5	6	7	8	9	WGBS	RRBS	MBD-BS
0	scaffold1_cov55	49	51	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
1	scaffold1_cov55	84	86	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
2	scaffold1_cov55	92	94	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
3	scaffold1_cov55	102	104	16.666667	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	8.3333335	N/A	N/A
4	scaffold1_cov55	105	107	0.0	12.5	N/A	N/A	N/A	N/A	N/A	N/A	N/A	6.25	N/A	N/A
5	scaffold1_cov55	116	118	0.0	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
6	scaffold1_cov55	119	121	0.0	0.0	20.0	N/A	N/A	N/A	N/A	N/A	N/A	6.666666666666667	N/A	N/A
7	scaffold1_cov55	146	148	0.0	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A
8	scaffold1_cov55	169	171	N/A	0.0	N/A	N/A	N/A	N/A	N/A	N/A	N/A	0.0	N/A	N/A


In [74]:
#Remove header
#Keep chr, start, end, and WGBS average (col 2-4, 13)
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Pact_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $14}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Pact_union_5x-averages-WGBS.bedgraph

In [75]:
#Check output: chr, start, end, % meth
!head Pact_union_5x-averages-WGBS.bedgraph
!wc -l Pact_union_5x-averages-WGBS.bedgraph

scaffold1_cov55	49	51	0.0
scaffold1_cov55	84	86	0.0
scaffold1_cov55	92	94	0.0
scaffold1_cov55	102	104	8.3333335
scaffold1_cov55	105	107	6.25
scaffold1_cov55	116	118	0.0
scaffold1_cov55	119	121	6.666666666666667
scaffold1_cov55	146	148	0.0
scaffold1_cov55	169	171	0.0
scaffold1_cov55	186	188	10.0
 7665143 Pact_union_5x-averages-WGBS.bedgraph


In [76]:
#Remove header
#Keep chr, start, end, and RRBS average
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Pact_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $15}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Pact_union_5x-averages-RRBS.bedgraph

In [77]:
#Check output: chr, start, end, % meth
!head Pact_union_5x-averages-RRBS.bedgraph
!wc -l Pact_union_5x-averages-RRBS.bedgraph

scaffold6_cov64	2536	2538	0.0
scaffold6_cov64	2584	2586	0.0
scaffold6_cov64	2676	2678	48.888889
scaffold6_cov64	2682	2684	0.0
scaffold6_cov64	4553	4555	17.142857
scaffold6_cov64	4588	4590	7.407407333333333
scaffold6_cov64	5101	5103	0.0
scaffold6_cov64	5309	5311	0.0
scaffold6_cov64	5456	5458	0.0
scaffold6_cov64	5486	5488	0.0
 3508578 Pact_union_5x-averages-RRBS.bedgraph


In [78]:
#Remove header
#Keep chr, start, end, and MBD-BS average
#Remove rows where the 4th column (average %meth) is N/A
#Save file
!tail -n +2 Pact_union_5x-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{print $2, $3, $4, $16}' \
| awk -F'\t' -v OFS='\t' '$4 != "N/A"' \
> Pact_union_5x-averages-MBDBS.bedgraph

In [79]:
#Check output: chr, start, end, % meth
!head Pact_union_5x-averages-MBDBS.bedgraph
!wc -l Pact_union_5x-averages-MBDBS.bedgraph

scaffold2_cov51	649	651	0.0
scaffold2_cov51	686	688	0.0
scaffold2_cov51	778	780	0.0
scaffold3_cov83	118	120	37.142857
scaffold3_cov83	130	132	26.25
scaffold3_cov83	137	139	37.5
scaffold3_cov83	189	191	41.452991
scaffold3_cov83	208	210	42.12885166666667
scaffold3_cov83	243	245	0.0
scaffold3_cov83	261	263	47.75132266666666
 5906529 Pact_union_5x-averages-MBDBS.bedgraph


In [80]:
!find *averages-*bedgraph

Pact_union_5x-averages-MBDBS.bedgraph
Pact_union_5x-averages-RRBS.bedgraph
Pact_union_5x-averages-WGBS.bedgraph


In [81]:
!wc -l *averages-*bedgraph > Pact_union_5x-averages-counts.txt

### 2. Characterize methylation for each CpG dinucleotide

- Methylated: > 50% methylation
- Sparsely methylated: 10-50% methylation
- Unmethylated: < 10% methylation

##### Methylated loci

In [82]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-Meth
done

In [83]:
!head *Meth

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth <==
scaffold3_cov83 475 477 61.45454566666666
scaffold3_cov83 504 506 51.6666665
scaffold7_cov100 5986 5988 77.14912266666668
scaffold7_cov100 6144 6146 96.29377666666666
scaffold7_cov100 6188 6190 95.25813700000002
scaffold7_cov100 6198 6200 95.69715500000001
scaffold7_cov100 6231 6233 78.51851866666668
scaffold7_cov100 6233 6235 95.98157266666665
scaffold7_cov100 7201 7203 80.0
scaffold7_cov100 7438 7440 80.0

==> Pact_union_5x-averages-RRBS.bedgraph-Meth <==
scaffold7_cov100 1535 1537 60.0
scaffold7_cov100 17000 17002 100.0
scaffold7_cov100 17090 17092 100.0
scaffold7_cov100 24454 24456 100.0
scaffold7_cov100 24494 24496 87.5
scaffold7_cov100 24509 24511 100.0
scaffold7_cov100 24557 24559 100.0
scaffold7_cov100 33140 33142 80.0
scaffold7_cov100 33157 33159 80.0
scaffold7_cov100 40896 40898 50.0

==> Pact_union_5x-averages-WGBS.bedgraph-Meth <==
scaffold7_cov100 5500 5502 77.77777766666667
scaffold7_cov100 5986 5988 75.55555566666668
sca

In [84]:
!wc -l *Meth

  725453 Pact_union_5x-averages-MBDBS.bedgraph-Meth
   66798 Pact_union_5x-averages-RRBS.bedgraph-Meth
  154042 Pact_union_5x-averages-WGBS.bedgraph-Meth
  946293 total


In [85]:
!wc -l *-Meth > Pact_union_5x-Meth-counts.txt

##### Sparsely methylated loci

In [86]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 < 50) { print $1, $2, $3, $4}}' ${f} \
    | awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
    > ${f}-sparseMeth
done

In [87]:
!head *sparseMeth

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth <==
scaffold3_cov83 118 120 37.142857
scaffold3_cov83 130 132 26.25
scaffold3_cov83 137 139 37.5
scaffold3_cov83 189 191 41.452991
scaffold3_cov83 208 210 42.12885166666667
scaffold3_cov83 261 263 47.75132266666666
scaffold3_cov83 484 486 47.386809
scaffold6_cov64 4146 4148 16.25
scaffold6_cov64 5904 5906 14.285714000000002
scaffold6_cov64 6880 6882 10.714285500000003

==> Pact_union_5x-averages-RRBS.bedgraph-sparseMeth <==
scaffold6_cov64 2676 2678 48.888889
scaffold6_cov64 4553 4555 17.142857
scaffold6_cov64 5904 5906 10.800086
scaffold6_cov64 6374 6376 24.382284333333335
scaffold6_cov64 7373 7375 16.666667
scaffold7_cov100 2301 2303 25.0
scaffold7_cov100 4351 4353 20.935810000000004
scaffold7_cov100 15408 15410 20.0
scaffold7_cov100 17074 17076 33.333333
scaffold7_cov100 17098 17100 20.0

==> Pact_union_5x-averages-WGBS.bedgraph-sparseMeth <==
scaffold1_cov55 252 254 20.0
scaffold2_cov51 686 688 11.609686333333334
scaffold3_cov83 

In [88]:
!wc -l *sparseMeth

  713277 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth
  210985 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth
  311665 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth
 1235927 total


In [89]:
!wc -l *-sparseMeth > Pact_union_5x-sparseMeth-counts.txt

##### Unmethylated loci

In [90]:
%%bash
for f in *averages-*bedgraph
do
    awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-unMeth
done

In [91]:
!head *unMeth

==> Pact_union_5x-averages-MBDBS.bedgraph-unMeth <==
scaffold2_cov51 649 651 0.0
scaffold2_cov51 686 688 0.0
scaffold2_cov51 778 780 0.0
scaffold3_cov83 243 245 0.0
scaffold6_cov64 290 292 0.0
scaffold6_cov64 298 300 0.0
scaffold6_cov64 489 491 0.0
scaffold6_cov64 826 828 7.142857000000001
scaffold6_cov64 2097 2099 0.0
scaffold6_cov64 2179 2181 0.0

==> Pact_union_5x-averages-RRBS.bedgraph-unMeth <==
scaffold6_cov64 2536 2538 0.0
scaffold6_cov64 2584 2586 0.0
scaffold6_cov64 2682 2684 0.0
scaffold6_cov64 4588 4590 7.407407333333333
scaffold6_cov64 5101 5103 0.0
scaffold6_cov64 5309 5311 0.0
scaffold6_cov64 5456 5458 0.0
scaffold6_cov64 5486 5488 0.0
scaffold6_cov64 5545 5547 4.761904666666667
scaffold6_cov64 5559 5561 3.952991666666667

==> Pact_union_5x-averages-WGBS.bedgraph-unMeth <==
scaffold1_cov55 49 51 0.0
scaffold1_cov55 84 86 0.0
scaffold1_cov55 92 94 0.0
scaffold1_cov55 102 104 8.3333335
scaffold1_cov55 105 107 6.25
scaffold1_cov55 116 118 0.0
scaffold1_cov55 119 121 6.666666

In [92]:
!wc -l *unMeth

 4467799 Pact_union_5x-averages-MBDBS.bedgraph-unMeth
 3230795 Pact_union_5x-averages-RRBS.bedgraph-unMeth
 7199436 Pact_union_5x-averages-WGBS.bedgraph-unMeth
 14898030 total


In [93]:
!wc -l *-unMeth > Pact_union_5x-unMeth-counts.txt

### 3. Characterize genomic locations of CpGs

#### 3a. Create BEDfiles

In [94]:
%%bash

for f in *averages-*bedgraph*
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

 5906529 Pact_union_5x-averages-MBDBS.bedgraph.bed
  725453 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed
  713277 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed
 4467799 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed
 3508578 Pact_union_5x-averages-RRBS.bedgraph.bed
   66798 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed
  210985 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed
 3230795 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed
 7665143 Pact_union_5x-averages-WGBS.bedgraph.bed
  154042 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed
  311665 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed
 7199436 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed


In [95]:
#Confirm file creation
!head Pact_union_5x-averages-MBDBS.bedgraph.bed

scaffold2_cov51	649	651
scaffold2_cov51	686	688
scaffold2_cov51	778	780
scaffold3_cov83	118	120
scaffold3_cov83	130	132
scaffold3_cov83	137	139
scaffold3_cov83	189	191
scaffold3_cov83	208	210
scaffold3_cov83	243	245
scaffold3_cov83	261	263


#### 3b. Genes

In [96]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.Genes.gff \
  > ${f}-paGenes
done

In [97]:
#Check output
!head *paGenes

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paGenes <==
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	7201	7203	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7696	7698	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7796	7798	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7891	7893	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	8323	8325	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paGenes <==
scaffold6_cov64	4146	4148	scaffold6_cov64	AUGUSTUS	gene	1	5652	0.46	-	.	g1
scaffold6_

In [98]:
#Count number of overlaps
!wc -l *paGenes

  328782 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paGenes
  286913 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paGenes
 2135588 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paGenes
 2751283 Pact_union_5x-averages-MBDBS.bedgraph.bed-paGenes
   28818 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paGenes
   85395 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paGenes
 1359026 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paGenes
 1473239 Pact_union_5x-averages-RRBS.bedgraph.bed-paGenes
  100847 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paGenes
  119765 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paGenes
 2981190 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paGenes
 3201802 Pact_union_5x-averages-WGBS.bedgraph.bed-paGenes
 14852648 total


In [99]:
!wc -l *paGenes > Pact_union_5x-paGenes-counts.txt

#### 3c. Coding Sequences (CDS)

In [100]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.CDS.gff \
  > ${f}-paCDS
done

In [101]:
#Check output
!head *paCDS

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paCDS <==
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	7696	7698	scaffold7_cov100	AUGUSTUS	CDS	7650	7715	1	-	2	transcript_id "g5.t1"; gene_id "g5";
scaffold7_cov100	7696	7698	scaffold7_cov100	AUGUSTUS	CDS	7650	7715	1	-	2	transcript_id "g5.t2"; gene_id "g5";
scaffold7_cov100	7892	7893	scaffold7_cov1

In [102]:
#Count number of overlaps
!wc -l *paCDS

  257900 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paCDS
  207579 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paCDS
 1571014 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paCDS
 2036493 Pact_union_5x-averages-MBDBS.bedgraph.bed-paCDS
   19990 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paCDS
   55457 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paCDS
  872936 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paCDS
  948383 Pact_union_5x-averages-RRBS.bedgraph.bed-paCDS
   76488 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paCDS
   64203 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paCDS
 1688071 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paCDS
 1828762 Pact_union_5x-averages-WGBS.bedgraph.bed-paCDS
 9627276 total


In [103]:
!wc -l *paCDS > Pact_union_5x-paCDS-counts.txt

#### 3d. Introns

In [104]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.Intron.gff \
  > ${f}-paIntron
done

In [105]:
#Check output
!head *paIntron

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paIntron <==
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	7201	7203	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t1"; gene_id "g5";
scaffold7_cov100	7201	7203	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t2"; gene_id "g5";
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t1"; gene_id "g5";
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t2"; gene_id "g5";
scaffold7_cov100	7796	7798	scaffold7_cov100	AUGUSTUS	intron	7716	7892	1	-	.	transcript_id "g5.t1"; gene_id "g5";
scaffold7_cov100	7796	7798	scaffold7_cov100	AUGUSTUS	intron	7716	7892	1	-	.	transcript_id "g5.t2"; gene_id "g5";
scaffold7_cov100	7891	7892	scaff

In [106]:
#Count number of overlaps
!wc -l *paIntron

  178614 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paIntron
  169278 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paIntron
 1292323 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paIntron
 1640215 Pact_union_5x-averages-MBDBS.bedgraph.bed-paIntron
   18551 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paIntron
   58617 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paIntron
  945665 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paIntron
 1022833 Pact_union_5x-averages-RRBS.bedgraph.bed-paIntron
   61765 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paIntron
   95770 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paIntron
 2335519 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paIntron
 2493054 Pact_union_5x-averages-WGBS.bedgraph.bed-paIntron
 10312204 total


In [107]:
!wc -l *paIntron > Pact_union_5x-paIntron-counts.txt

#### 3e. Flanking regions

In [108]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.flanks.gff \
  > ${f}-paFlanks
done

In [109]:
#Check output
!head *paFlanks

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanks <==
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	11815	11817	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	12652	12654	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	12652	12654	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12662	12664	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	12662	12664	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12675	12677	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanks <==
scaffold6_cov64	6880	6882	scaffold6_cov64	AUGUSTUS

In [110]:
#Count number of overlaps
!wc -l *paFlanks

  174343 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanks
  172752 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanks
 1057105 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paFlanks
 1404200 Pact_union_5x-averages-MBDBS.bedgraph.bed-paFlanks
   16288 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paFlanks
   52827 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paFlanks
  846983 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paFlanks
  916098 Pact_union_5x-averages-RRBS.bedgraph.bed-paFlanks
   39523 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paFlanks
   78739 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paFlanks
 1856136 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paFlanks
 1974398 Pact_union_5x-averages-WGBS.bedgraph.bed-paFlanks
 8589392 total


In [111]:
!wc -l *paFlanks > Pact_union_5x-paFlanks-counts.txt

#### 3f. Upstream flanking regions

In [112]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.flanks.Upstream.gff \
  > ${f}-paFlanksUpstream
done

In [113]:
#Check output
!head *paFlanksUpstream

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanksUpstream <==
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	11815	11817	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	12652	12654	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	12662	12664	scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	18520	18522	scaffold7_cov100	AUGUSTUS	gene	18321	18585	0.74	+	.	g10
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.99	-	.	g9
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.74	+	.	g10
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.39	+	.	g11
scaffold7_cov100	19296	19298	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.99	-	.	g9

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanksUpstream <==
scaffold7_cov100	

In [114]:
#Count number of overlaps
!wc -l *paFlanksUpstream

   82907 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanksUpstream
   88959 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanksUpstream
  590141 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paFlanksUpstream
  762007 Pact_union_5x-averages-MBDBS.bedgraph.bed-paFlanksUpstream
    8527 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paFlanksUpstream
   28558 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paFlanksUpstream
  475159 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paFlanksUpstream
  512244 Pact_union_5x-averages-RRBS.bedgraph.bed-paFlanksUpstream
   18248 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paFlanksUpstream
   38615 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paFlanksUpstream
  997092 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paFlanksUpstream
 1053955 Pact_union_5x-averages-WGBS.bedgraph.bed-paFlanksUpstream
 4656412 total


In [115]:
!wc -l *paFlanksUpstream > Pact_union_5x-paFlanksUpstream-counts.txt

#### 3g. Downstream flanking regions

In [116]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.flanks.Downstream.gff \
  > ${f}-paFlanksDownstream
done

In [117]:
#Check output
!head *paFlanksDownstream

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanksDownstream <==
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	12652	12654	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12662	12664	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12675	12677	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12683	12685	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12704	12706	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12726	12728	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12737	12739	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	12806	12808	scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanksDownstream <==
scaffold6_cov64	688

In [118]:
#Count number of overlaps
!wc -l *paFlanksDownstream

   91436 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paFlanksDownstream
   83793 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paFlanksDownstream
  466964 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paFlanksDownstream
  642193 Pact_union_5x-averages-MBDBS.bedgraph.bed-paFlanksDownstream
    7761 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paFlanksDownstream
   24269 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paFlanksDownstream
  371824 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paFlanksDownstream
  403854 Pact_union_5x-averages-RRBS.bedgraph.bed-paFlanksDownstream
   21275 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paFlanksDownstream
   40124 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paFlanksDownstream
  859044 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paFlanksDownstream
  920443 Pact_union_5x-averages-WGBS.bedgraph.bed-paFlanksDownstream
 3932980 total


In [119]:
!wc -l *paFlanksDownstream > Pact_union_5x-paFlanksDownstream-counts.txt

#### 3h. Intergenic

In [120]:
%%bash 

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.intergenic.bed \
  > ${f}-paIntergenic
done

In [121]:
#Check output
!head *paIntergenic

==> Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paIntergenic <==
scaffold3_cov83	475	477	scaffold3_cov83	0	598
scaffold3_cov83	504	506	scaffold3_cov83	0	598
scaffold7_cov100	216293	216295	scaffold7_cov100	214550	217636
scaffold7_cov100	230584	230586	scaffold7_cov100	220023	234761
scaffold7_cov100	267363	267365	scaffold7_cov100	263796	267853
scaffold7_cov100	273435	273437	scaffold7_cov100	272559	273794
scaffold7_cov100	304579	304581	scaffold7_cov100	304168	309429
scaffold7_cov100	304583	304585	scaffold7_cov100	304168	309429
scaffold7_cov100	304590	304592	scaffold7_cov100	304168	309429
scaffold7_cov100	304601	304603	scaffold7_cov100	304168	309429

==> Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paIntergenic <==
scaffold3_cov83	118	120	scaffold3_cov83	0	598
scaffold3_cov83	130	132	scaffold3_cov83	0	598
scaffold3_cov83	137	139	scaffold3_cov83	0	598
scaffold3_cov83	189	191	scaffold3_cov83	0	598
scaffold3_cov83	208	210	scaffold3_cov83	0	598
scaffold3_cov83	261	263	scaffold3_cov83

In [122]:
#Count number of overlaps
!wc -l *paIntergenic

  280338 Pact_union_5x-averages-MBDBS.bedgraph-Meth.bed-paIntergenic
  304903 Pact_union_5x-averages-MBDBS.bedgraph-sparseMeth.bed-paIntergenic
 1528019 Pact_union_5x-averages-MBDBS.bedgraph-unMeth.bed-paIntergenic
 2113260 Pact_union_5x-averages-MBDBS.bedgraph.bed-paIntergenic
   26043 Pact_union_5x-averages-RRBS.bedgraph-Meth.bed-paIntergenic
   85523 Pact_union_5x-averages-RRBS.bedgraph-sparseMeth.bed-paIntergenic
 1226877 Pact_union_5x-averages-RRBS.bedgraph-unMeth.bed-paIntergenic
 1338443 Pact_union_5x-averages-RRBS.bedgraph.bed-paIntergenic
   26172 Pact_union_5x-averages-WGBS.bedgraph-Meth.bed-paIntergenic
  134445 Pact_union_5x-averages-WGBS.bedgraph-sparseMeth.bed-paIntergenic
 2782628 Pact_union_5x-averages-WGBS.bedgraph-unMeth.bed-paIntergenic
 2943245 Pact_union_5x-averages-WGBS.bedgraph.bed-paIntergenic
 12789896 total


In [123]:
!wc -l *paIntergenic > Pact_union_5x-paIntergenic-counts.txt